# Δεδομένα Πειράματος και Διαλογή Εικόνων
## Ορισμός των μεταβλητών συστήματος 
1. Οι εικόνες βρίσκονται στο dir `/data/raw`
2. το notebook `01-data_access.ipynb` βρίσκεται στο `/notebooks`
3. οπότε για να έχουμε πρόσβαση πρέπει να μεταφερθούμε στο root project directory 
### η δομή του φακέλου του κώδικα είναι
- src
    - data
    - features
    - models
    - visualization

In [1]:
# καταχωρούμε αν δεν έχουμε ήδη στο σύστημα το directory του project  
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
 

### έχουμε πρόσβαση σε ό,τι αρχεία έχουμε γράψει στο `src` πχ 
`from src.visualization import gif_maker`
### από ένα bash terminal εκτελούμε εντολή διαγραφής στο `/data/interim`
`$ make clean_interim`

### Eπεξεργαζόμαστε μία φορά τη βάση των εικόνων `/data/raw`
### Αποθηκεύουμε με την ίδια δομή στο φάκελο `/data/interim`
### Ο κώδικας για αυτό βρίσκεται στο `/src/data`

In [ ]:
from src.data import initial_data_pick
initial_data_pick()

In [ ]:
## Mετασχηματισμoί στα αρχικά δεδομένα 
### Θερμικές Εικόνες jpeg 
### Κανονικές Εικόνες jpeg 
### Τιμές Θερμοκρασιών csv 
